In [1]:
##install
%pip install bs4



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install lxml


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#imort
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:

url = "https://www.iisermohali.ac.in/people#Faculty"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
html = response.text

print(html[:500])

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en-gb" lang="en-gb" dir="ltr">
	<head>
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="viewport" content="width=device-width, initial-scale=1">
		<meta charset="utf-8">
	<meta name="generator" content="Joomla! - Open Source Content Management">
	<title>People</title>
	<link href="/images/favicons_iiser/favicon-16x16.png#joomlaImage://local-images/favicons_iiser/favicon-16x16.png?width=16&height=16" rel="ic


In [6]:
soup = BeautifulSoup(html, "lxml")

In [7]:
rows = soup.select("tbody tr")
print(f"Found {len(rows)} rows")


Found 362 rows


In [8]:
faculty_list = []
base_url = "https://www.iisermohali.ac.in/faculty/"   

for row in rows:
    cells = row.find_all("td")
    
    # Name + profile link (first cell)
    name_tag = cells[0].select_one("a") if len(cells) > 0 else None
    name = name_tag.get_text(strip=True) if name_tag else ""
    
    # Check if 'href' exists
    profile = name_tag["href"] if name_tag and name_tag.has_attr("href") else ""
    
    # Title (second cell)
    title = cells[1].get_text(strip=True) if len(cells) > 1 else ""
    
    # Email (third cell)
    email_tag = cells[2].select_one("a[href^='mailto:']") if len(cells) > 2 else None
    email = email_tag["href"].replace("mailto:", "") if email_tag else ""
    
    # Save
    faculty_list.append({
        "name": name,
        "profile": profile,
        "title": title,
        "email": email
    })

print(faculty_list[:3])

for faculty in faculty_list:
    profile_url = faculty["profile"]
    if profile_url:
        if profile_url.startswith("/"):
            profile_url = base_url + profile_url

        try:
            resp = requests.get(profile_url, timeout=10)
            soup = BeautifulSoup(resp.text, "lxml")

            # Extract PhD info
            phd_text = ""
            for p in soup.select(".sppb-addon-content p"):
                if "Ph.D." in p.get_text():
                    phd_text = p.get_text(strip=True)
            for p in soup.select(".sppb-addon-content p"):
                if "PhD" in p.get_text():
                    phd_text = p.get_text(strip=True)


            # Extract Postdoc info (optional, if present)
            postdoc_text = ""
            for p in soup.select(".sppb-addon-content p"):
                if "Postdoc" in p.get_text():
                    postdoc_text = p.get_text(strip=True)

            faculty["phd"] = phd_text
            faculty["postdoc"] = postdoc_text

        except Exception as e:
            print(f"Error fetching {profile_url}: {e}")
            faculty["phd"] = ""
            faculty["postdoc"] = ""

    else:
        faculty["phd"] = ""
        faculty["postdoc"] = ""


[{'name': '', 'profile': '', 'title': '', 'email': ''}, {'name': 'NIRF', 'profile': '/nirf-2', 'title': 'RTI', 'email': ''}, {'name': '', 'profile': '', 'title': 'FACULTYDIRECTORY SEARCH', 'email': ''}]


In [9]:
df = pd.DataFrame(faculty_list)
df.to_csv("faculty_with_phd.csv", index=False, encoding="utf-8")

print("✅ Data saved to faculty_with_phd.csv")

✅ Data saved to faculty_with_phd.csv
